In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Get the same results each time
np.random.seed(0)

In [5]:
# Load the training data
data = pd.read_csv("/content/drive/MyDrive/UrfuMagistracy/NNEthics/data.csv")
comments = data["comment_text"]
target = (data["target"]>0.7).astype(int)

In [6]:
comments.shape

(90902,)

### Task 1

In [7]:
X_train, X_test, y_train, y_test = train_test_split(comments, target, test_size=0.3, random_state=42)

### Task 2

In [8]:
VECTORIZER = CountVectorizer()

In [9]:
X_train_numeric = VECTORIZER.fit_transform(X_train)

In [10]:
X_train.shape

(63631,)

In [11]:
X_test_numeric = VECTORIZER.transform(X_test)

In [12]:
X_test.shape

(27271,)

### Task 3

In [13]:
model = LogisticRegression(max_iter=2000)

In [14]:
model.fit(X_train_numeric, y_train)

LogisticRegression(max_iter=2000)

In [15]:
y_pred = model.predict(X_test_numeric)

In [16]:
accuracy = accuracy_score(y_test, y_pred)

In [17]:
accuracy

0.9276887536210626

### Task 4

In [18]:
def toxic_comment_check(model, vectorizer, comment_string):
    vectorized_string = vectorizer.transform([comment_string])
    return model.predict_proba(vectorized_string)[:, 1][0]

### Task 5

In [19]:
toxic_comment_check(model, VECTORIZER, "Apples are stupid")

0.9990974774374046

In [20]:
toxic_comment_check(model, VECTORIZER, "I love apples")

0.060041641687158455

### Task 6

In [21]:
word_indices = list(VECTORIZER.vocabulary_.keys())

In [22]:
toxic_coefficients = model.coef_[0]

In [23]:
word_coefficients = {word: coefficient for word, coefficient in zip(word_indices, toxic_coefficients)}

In [24]:
most_toxic_words = sorted(word_coefficients.items(), key=lambda x: x[1], reverse=True)[:10]

In [25]:
for word, coefficient in most_toxic_words:
    print(f"{word}: {coefficient}")

yoy: 9.185995686456975
tokaido: 8.683573518776607
offensiveness: 8.42041305064192
anoints: 7.527125837942067
ei: 6.796410544575449
evolves: 6.5589103487913745
rings: 6.436969131849324
spunk: 6.396045203387988
doo: 6.373262348064005
benner: 6.349951150415368


### Task 7

#### Слова которых не должно быть здесь: yoy, tokaido, anoints, evolves, rings, benner

### Task 8

In [26]:
toxic_comment_check(model, VECTORIZER, "I have a christian friend")

0.18762460205687848

In [27]:
toxic_comment_check(model, VECTORIZER, "I have a muslim friend")

0.5091230690048197

In [28]:
toxic_comment_check(model, VECTORIZER, "I have a white friend")

0.40048201935060884

In [29]:
toxic_comment_check(model, VECTORIZER, "I have a black friend")

0.5879799245351816

### Task 9

In [30]:
religious_context_comments = [
    "I am a christian",
    "I am a muslim",
    "I am a buddhist",
    "I am a jewish",
    "I am a hindu",
    "I am an atheist",
    "I am an heathen",
]

print("Religious comments toxicity prediction")
for comment in religious_context_comments:
    print(f"'{comment}': {toxic_comment_check(model, VECTORIZER, comment)}")

Religious comments toxicity prediction
'I am a christian': 0.17172630848662204
'I am a muslim': 0.48215109717991717
'I am a buddhist': 0.18349838541122326
'I am a jewish': 0.2656821878793225
'I am a hindu': 0.0927159070702703
'I am an atheist': 0.2696733145223526
'I am an heathen': 0.24333336296583427


### Task 10

1. Нейтрализовать религиозные индикаторы: удалить или нейтрализовать явные религиозные индикаторы из комментариев перед тем, как отправить их в алгоритм. Это поможет уменьшить предвзятость на основе религиозных упоминаний.

2. Ориентироваться на поведение, а не на убеждения: вместо того чтобы напрямую анализировать комментарии, связанные с религией, сосредоточиться на анализе поведения или действий, отраженных в комментариях.

### Task 11

 1 Сбор и разметка данных: собрать достаточное количество комментариев, которые будут размечены как токсичные и нетоксичные.

 2 Предварительная обработка текста: очистить комментарии от лишних символов, стоп-слов и проведите лемматизацию или стемминг для нормализации текста.

 3 Извлечение признаков: преобразовать комментарии в числовые признаки, которые модель сможет использовать для классификации. Это может включать использование методов, таких как мешок слов (bag-of-words), TF-IDF или эмбеддинги, такие как Word2Vec или GloVe.

 4 Выбор модели машинного обучения: выбрать подходящую модель машинного обучения, такую как логистическая регрессия, метод опорных векторов (SVM) или нейронная сеть. Обучить модель.

 5 Оценка модели: оценить производительность модели с использованием метрик, таких как точность, полнота и F1-мера. При необходимости провести кросс-валидацию и настройку гиперпараметров модели.

 6 Регулярно обновлять модель: периодически обновлять модель с использованием новых данных, чтобы она оставалась актуальной и этично классифицировала комментарии.